In [2]:
import pandas as pd

## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [3]:
r_df = pd.read_csv('ratings.csv')
r_df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [45]:
top_users = r_df.groupby('userId').count()
top_users = top_users[(top_users.rating > 100)].drop(['movieId', 'timestamp'], axis=1)

users_timestamp = r_df.groupby('userId').agg({'timestamp': ['min', 'max']})
users_timestamp['lifetime'] = users_timestamp['timestamp']['max'] - users_timestamp['timestamp']['min']

# # меняю местами уровни названий столбцов
# users_timestamp.columns = users_timestamp.columns.swaplevel(0, 1)
# # разворачиваю порядок названий
# users_timestamp.sort_index(axis=1, level=0, inplace=True)
# # удаляю названия столбцов верхнего уровня
# users_timestamp.columns = users_timestamp.columns.droplevel(0)

# объеднияю названия столбцов обоих уровней
users_timestamp.columns = ['_'.join(col) for col in users_timestamp.columns]


users_timestamp.head()

,timestamp_min,timestamp_max,lifetime_
userId,,,
1,1260759108,1260759205,97
2,835355395,835356246,851
3,1298861589,1298932787,71198
4,949778714,949982274,203560
5,1163373044,1163375145,2101


In [46]:
users_lifetime = top_users.join(users_timestamp)

print('5-ка с наименьшим количеством выставленных оценок:')
users_lifetime.sort_values('rating', ascending = False).tail()

5-ка с наименьшим количеством выставленных оценок:


,rating,timestamp_min,timestamp_max,lifetime_
userId,,,,
217,104,1108160000,1108160981,981
36,104,847056510,853005800,5949290
623,103,1225255417,1225258519,3102
67,103,854711770,854715906,4136
257,103,1338006929,1449941803,111934874


## Задание 2

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

Необходимо сформировать две таблицы:

* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

* аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [47]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)


auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)


air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)


client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [48]:
full_df = rzd.merge(auto, how='outer').merge(air, how='outer').fillna(0).set_index('client_id')
# full_df.set_index('client_id')
full_df['total_by_client'] = full_df.sum(axis=1)
full_df.loc['total_by_type']= full_df.sum()
full_df

,rzd_revenue,auto_revenue,air_revenue,total_by_client
client_id,,,,
111,1093.0,0.0,0.0,1093.0
112,2810.0,0.0,0.0,2810.0
113,10283.0,57483.0,0.0,67766.0
114,5774.0,83.0,0.0,5857.0
115,981.0,912.0,81.0,1974.0
116,0.0,4834.0,4.0,4838.0
117,0.0,98.0,13.0,111.0
118,0.0,0.0,173.0,173.0
total_by_type,20941.0,63410.0,271.0,84622.0


In [49]:
# не уверен, что правильно понял задачу:

client_base.set_index('client_id').join(full_df)

,address,rzd_revenue,auto_revenue,air_revenue,total_by_client
client_id,,,,,
111,Комсомольская 4,1093.0,0.0,0.0,1093.0
112,Энтузиастов 8а,2810.0,0.0,0.0,2810.0
113,Левобережная 1а,10283.0,57483.0,0.0,67766.0
114,Мира 14,5774.0,83.0,0.0,5857.0
115,ЗЖБИиДК 1,981.0,912.0,81.0,1974.0
116,Строителей 18,0.0,4834.0,4.0,4838.0
117,Панфиловская 33,0.0,98.0,13.0,111.0
118,Мастеркова 4,0.0,0.0,173.0,173.0
